<a href="https://colab.research.google.com/github/ThamayanthiM/Assignment_task/blob/main/Assignment_fine_tuned_GPT_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Generation

In [ ]:
prompt = "A model that takes in a user's skills, interests, and preferred industries and provides tailored career advice, including possible job roles, skills to develop, and resources for further learning."
temperature = .3
number_of_examples = 50

Run this to generate the dataset.

In [ ]:
!pip install openai==0.28


In [ ]:
import os
import openai
import random
from tenacity import retry, stop_after_attempt, wait_exponential

openai.api_key = "YOUR API KEY"

N_RETRIES = 3

@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def generate_example(prompt, prev_examples, temperature=.3):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 8:
            prev_examples = random.sample(prev_examples, 8)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1000,
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
Generating example 5
Generating example 6
Generating example 7
Generating example 8
Generating example 9
Generating example 10
Generating example 11
Generating example 12
Generating example 13
Generating example 14
Generating example 15
Generating example 16
Generating example 17
Generating example 18
Generating example 19
Generating example 20
Generating example 21
Generating example 22
Generating example 23
Generating example 24
Generating example 25
Generating example 26
Generating example 27
Generating example 28
Generating example 29
Generating example 30
Generating example 31
Generating example 32
Generating example 33
Generating example 34
Generating example 35
Generating example 36
Generating example 37
Generating example 38
Generating example 39
Generating example 40
Generating example 41
Generating example 42
Generating example 43
Generating example 44
Generating example 4

# We also need to generate a system message.

In [ ]:
def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given your skills, interests, and preferred industries, the model will provide tailored career advice, including possible job roles, skills to develop, and resources for further learning.`. Feel free to re-run this cell if you want a better result.


Now let's put our examples into a dataframe and turn them into a final pair of datasets.

In [ ]:
import json
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples.')

# Initialize list to store training examples
training_examples = []

# Create training examples in the format required for GPT-3.5 fine-tuning
for index, row in df.iterrows():
    training_example = {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    training_examples.append(training_example)

# Save training examples to a .jsonl file
with open('training_examples.jsonl', 'w') as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')

There are 49 successfully-generated examples.


# Upload the file to OpenAI

In [ ]:
file_id = openai.File.create(
  file=open("/content/training_examples.jsonl", "rb"),
  purpose='fine-tune'
).id

print(f'File uploaded successfully! The file ID is: {file_id}')

File uploaded successfully! The file ID is: file-I7Qg5Wth74xBCv0GrMnUaHLp


# Train the model! You may need to wait a few minutes before running the next cell to allow for the file to process on OpenAI's servers.

In [ ]:
job = openai.FineTuningJob.create(training_file=file_id, model="gpt-3.5-turbo")

job_id = job.id

In [ ]:
# Retrieve the status of the fine-tuning job
status = openai.FineTuningJob.retrieve(id=job_id)
print(status)

{
  "object": "fine_tuning.job",
  "id": "ftjob-46WhkclGwhtXRjHYZYFak26u",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1726808855,
  "finished_at": 1726809296,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0125:flipick::A9QGosxf",
  "organization_id": "org-CZPNgOwyHPUGaC7iFdns5LJZ",
  "result_files": [
    "file-mtR1JSwMC9bcOuLubTeLosm1"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-I7Qg5Wth74xBCv0GrMnUaHLp",
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 27156,
  "error": {},
  "user_provided_suffix": null,
  "seed": 1445945002,
  "estimated_finish": null,
  "integrations": []
}


# Now, just wait until the fine-tuning run is done, and you'll have a ready-to-use model!



In [ ]:
events = openai.FineTuningJob.list_events(id=job_id, limit=10)
# Print the events
for event in events['data']:
    print(f"Event Type: {event['type']}, Timestamp: {event['created_at']}, Message: {event['message']}")

Event Type: message, Timestamp: 1726809301, Message: The job has successfully completed
Event Type: message, Timestamp: 1726809298, Message: New fine-tuned model created
Event Type: message, Timestamp: 1726809298, Message: Checkpoint created at step 98
Event Type: message, Timestamp: 1726809298, Message: Checkpoint created at step 49
Event Type: metrics, Timestamp: 1726809282, Message: Step 147/147: training loss=0.15
Event Type: metrics, Timestamp: 1726809275, Message: Step 146/147: training loss=0.23
Event Type: metrics, Timestamp: 1726809271, Message: Step 145/147: training loss=0.21
Event Type: metrics, Timestamp: 1726809271, Message: Step 144/147: training loss=0.10
Event Type: metrics, Timestamp: 1726809271, Message: Step 143/147: training loss=0.22
Event Type: metrics, Timestamp: 1726809271, Message: Step 142/147: training loss=0.13


In [ ]:
events

<OpenAIObject list at 0x7b42a6233ec0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-JyVtOK1eh8GNNXPNEJfwoAdy",
      "created_at": 1726809301,
      "level": "info",
      "message": "The job has successfully completed",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-1Te4zzs21LrdH1PEzYTiEVu5",
      "created_at": 1726809298,
      "level": "info",
      "message": "New fine-tuned model created",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-obSB89jQck4aqiXgHcrOPArC",
      "created_at": 1726809298,
      "level": "info",
      "message": "Checkpoint created at step 98",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-juadxginlYdADrdPgYdkNbTn",
      "created_at": 1726809298,
      "level": "info",
      "message": "Che

# Once your model is trained, run the next cell to grab the fine-tuned model name.

In [ ]:
model_name_pre_object = openai.FineTuningJob.retrieve(job_id)
model_name = model_name_pre_object.fine_tuned_model
print(model_name)

ft:gpt-3.5-turbo-0125:flipick::A9QGosxf


# Let's try it out!

In [ ]:
response = openai.ChatCompletion.create(
    model=model_name,
    messages=[
      {
        "role": "system",
        "content": system_message,
      },
      {
          "role": "user",
          "content": df['prompt'].sample().values[0],
      }
    ],
)

response.choices[0].message['content']

'Given your skills and interests, you might consider a career as a Talent Acquisition Specialist, HR Business Partner, or Organizational Development Consultant in the tech and startup industry. To further develop your skills, consider learning more about data-driven HR, diversity and inclusion, and change management. Resources for further learning include online courses on platforms like Coursera, edX, or LinkedIn Learning. Joining HR professional networks or attending industry conferences can also provide valuable networking and learning opportunities.'